In [2]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import os

# Configurar el generador de números aleatorios para reproducibilidad
np.random.seed(42)

# Generar fechas para cinco años
start_date = datetime(2019, 1, 1)
end_date = datetime(2023, 12, 31)
dates = pd.date_range(start=start_date, end=end_date, freq='D')

# Definir productos y sus características base
productos = {
    'Laptop': {'base_price': 1000, 'volatility': 0.3, 'trend': 0.1},
    'Smartphone': {'base_price': 500, 'volatility': 0.4, 'trend': 0.2},
    'Tablet': {'base_price': 300, 'volatility': 0.3, 'trend': 0.05},
    'Smartwatch': {'base_price': 200, 'volatility': 0.5, 'trend': 0.15},
    'Auriculares': {'base_price': 100, 'volatility': 0.2, 'trend': 0.1}
}

# Función para generar ventas con estacionalidad y tendencia
def generate_sales(dates, product_info):
    sales = []
    for date in dates:
        base = product_info['base_price']
        day_of_year = date.dayofyear
        year = date.year - start_date.year
        
        # Tendencia anual
        trend = 1 + (year * product_info['trend'])
        
        # Estacionalidad
        seasonality = 1 + 0.3 * np.sin(2 * np.pi * day_of_year / 365)
        
        # Efecto de vacaciones (aumento en diciembre)
        holiday_effect = 1.5 if date.month == 12 else 1
        
        # Volatilidad diaria
        daily_volatility = np.random.normal(1, product_info['volatility'])
        
        sale = base * trend * seasonality * holiday_effect * daily_volatility
        sales.append(round(sale, 2))
    
    return sales

# Crear DataFrame
data = []
for producto, info in productos.items():
    ventas = generate_sales(dates, info)
    for date, venta in zip(dates, ventas):
        data.append({
            'fecha': date,
            'producto': producto,
            'ventas': venta
        })

df = pd.DataFrame(data)

# Guardar a CSV
csv_path = './datos/ventas_mensuales_extendido.csv'
os.makedirs(os.path.dirname(csv_path), exist_ok=True)
df.to_csv(csv_path, index=False)

print(f"Archivo '{csv_path}' generado exitosamente.")

# Verificar el contenido del archivo
try:
    df_check = pd.read_csv(csv_path)
    print("\nContenido del archivo (primeras 5 filas):")
    print(df_check.head())
    print(f"\nDimensiones del DataFrame: {df_check.shape}")
    print("\nResumen estadístico:")
    print(df_check.describe())
except Exception as e:
    print(f"Error al leer el archivo: {e}")

# Verificar la existencia y el tamaño del archivo
if os.path.exists(csv_path):
    file_size = os.path.getsize(csv_path)
    print(f"\nEl archivo existe y su tamaño es de {file_size} bytes.")
else:
    print(f"\nEl archivo no existe en la ruta especificada: {csv_path}")


Archivo './datos/ventas_mensuales_extendido.csv' generado exitosamente.

Contenido del archivo (primeras 5 filas):
        fecha producto   ventas
0  2019-01-01   Laptop  1154.95
1  2019-01-02   Laptop   968.42
2  2019-01-03   Laptop  1212.80
3  2019-01-04   Laptop  1486.98
4  2019-01-05   Laptop   953.73

Dimensiones del DataFrame: (9130, 3)

Resumen estadístico:
           ventas
count  9130.00000
mean    544.84139
std     507.67930
min    -213.77000
25%     167.63250
50%     357.22500
75%     773.26000
max    3346.78000

El archivo existe y su tamaño es de 251878 bytes.
